In [5]:
import face_recognition
import imutils
import pickle
import time
import cv2
import os
import numpy as np

In [6]:
# load the harcaascade in the cascade classifier
#new_path = r'C:\Users\LENOVO\Desktop\Ressources\Ressources\OpenCV\opencv-4.5.5\data\haarcascades'
face_cascade = cv2.CascadeClassifier(r'C:\Users\LENOVO\Desktop\Ressources\Ressources\OpenCV\opencv-4.5.5\data\haarcascades\haarcascade_frontalface_alt2.xml')
eye_cascade  = cv2.CascadeClassifier(r'C:\Users\LENOVO\Desktop\Ressources\Ressources\OpenCV\opencv-4.5.5\data\haarcascades\haarcascade_eye.xml')
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())

In [7]:
#Find path to the image you want to detect face and pass it here
image = cv2.imdecode(np.fromfile(str(r'C:\Users\LENOVO\Desktop\Ressources\OpenCV_Tensorflow\Face_recognition\test_picture.jpg'), dtype=np.uint8), cv2.IMREAD_UNCHANGED)
#image = cv2.imread(r'C:\Users\LENOVO\Desktop\Ressources\OpenCV_Tensorflow\Face_recognition\test_picture.jpg')
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#convert image to Greyscale for haarcascade
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray,
                                     scaleFactor=1.1,
                                     minNeighbors=5,
                                     minSize=(60, 60),
                                     flags=cv2.CASCADE_SCALE_IMAGE)

eyes = eye_cascade.detectMultiScale(gray)

In [8]:
# the facial embeddings for face in input
encodings = face_recognition.face_encodings(rgb)
names = []
# loop over the facial embeddings incase
# we have multiple embeddings for multiple fcaes
for encoding in encodings:
    #Compare encodings with encodings in data["encodings"]
    #Matches contain array with boolean values and True for the embeddings it matches closely
    #and False for rest
    matches = face_recognition.compare_faces(data["encodings"],
    encoding)
    #set name =inknown if no encoding matches
    name = "Unknown"
    # check to see if we have found a match
    if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)
 
 
        # update the list of names
        names.append(name)
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #roi_gray = gray[y:y+h, x:x+w]
            #roi_color = image[y:y+h, x:x+w]
            for ((a, b, c, d)) in eyes:
                cv2.rectangle(image, (a, b), (a + c, b + d), (0, 255, 0), 2)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
    cv2.imshow("Frame", image)
    cv2.waitKey(0)